In [1]:
# === ended with an optimal accuracy of ~59%, did not include as final model ===

In [2]:
# installing tensorflow to make it work in jupyter notebook
# (you can skip this cell if you already have tensorflow installed in your dev environment)
!pip install tensorflow

In [2]:
# import dependencies
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import numpy as np

# import movie csv
movies_df = pd.read_csv('Resources/movies_cleaned.csv')

# display df
movies_df.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [3]:
# create new revenue column
movies_df['percent_revenue'] = (movies_df['gross']/ movies_df['budget'])*100

# include only necessary columns
cleaned_movies_df = movies_df[['rating', 'year', 'genre', 'director', 'star', 'budget', 'percent_revenue', 'runtime']]

# display newly cleaned df
cleaned_movies_df.head()

,rating,year,genre,director,star,budget,percent_revenue,runtime
0,R,1980,Drama,Stanley Kubrick,Jack Nicholson,19000000.0,247.361958,146.0
1,R,1980,Adventure,Randal Kleiser,Brooke Shields,4500000.0,1307.846800,104.0
2,PG,1980,Action,Irvin Kershner,Mark Hamill,18000000.0,2990.972594,124.0
3,PG,1980,Comedy,Jim Abrahams,Robert Hays,3500000.0,2384.386829,88.0
4,R,1980,Comedy,Harold Ramis,Chevy Chase,6000000.0,664.105733,98.0


In [4]:
# display columns types
cleaned_movies_df.dtypes

rating              object
year                 int64
genre               object
director            object
star                object
budget             float64
percent_revenue    float64
runtime            float64
dtype: object

In [5]:
# encode non-numerical data
dummies = pd.get_dummies(data=cleaned_movies_df)

# combining the newly encoded data with the original dataframe
concat_df = pd.concat([cleaned_movies_df, dummies], axis=1)

# removing duplicate columns
concat_df = concat_df.loc[:,~concat_df.columns.duplicated()].copy()

#dropping the old, not encoded columns
concat_df = concat_df.drop(columns=['genre', 'director', 'star', 'rating'])

concat_df.head()

,year,budget,percent_revenue,runtime,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,...,star_Zachary Quinto,star_Zack Gottsagen,star_Zack Ward,star_Zain Al Rafeea,star_Zhi-zhong Huang,star_Ziyi Zhang,star_Zoe Margaret Colletti,star_Zoe Saldana,star_Zoey Deutch,star_Zooey Deschanel
0,1980,19000000.0,247.361958,146.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1980,4500000.0,1307.846800,104.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1980,18000000.0,2990.972594,124.0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,1980,3500000.0,2384.386829,88.0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,1980,6000000.0,664.105733,98.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# creating the profit_category column

# create bins for percentages
bins = [0, 200]

# create labels for bins
labels = np.array([0, 1])

# add new column
concat_df["profit_category"] = labels[np.searchsorted(bins, concat_df["percent_revenue"])-1]

# drop old percent_revenue column
concat_df = concat_df.drop(columns=['percent_revenue'])


concat_df.head()

,year,budget,runtime,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R,...,star_Zack Gottsagen,star_Zack Ward,star_Zain Al Rafeea,star_Zhi-zhong Huang,star_Ziyi Zhang,star_Zoe Margaret Colletti,star_Zoe Saldana,star_Zoey Deutch,star_Zooey Deschanel,profit_category
0,1980,19000000.0,146.0,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,1
1,1980,4500000.0,104.0,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,1
2,1980,18000000.0,124.0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,1
3,1980,3500000.0,88.0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,1
4,1980,6000000.0,98.0,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,1


In [8]:
# split our preprocessed data into our features and target arrays
y = concat_df['profit_category']
X = concat_df.drop(columns='profit_category')

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
# create a StandardScaler instances
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# define the model
model = tf.keras.models.Sequential()

# first hidden layer
model.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=3930))

# second hidden layer
model.add(tf.keras.layers.Dense(units=100, activation="relu"))

# output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               393100    
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 403301 (1.54 MB)
Trainable params: 403301 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train the model
fit_model = model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
127/127 [==============================] - 2s 6ms/step - loss: 0.7080 - accuracy: 0.5421
Epoch 2/100
127/127 [==============================] - 1s 6ms/step - loss: 0.3479 - accuracy: 0.8589
Epoch 3/100
127/127 [==============================] - 1s 5ms/step - loss: 0.1921 - accuracy: 0.9232
Epoch 4/100
127/127 [==============================] - 1s 6ms/step - loss: 0.1212 - accuracy: 0.9542
Epoch 5/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0898 - accuracy: 0.9665
Epoch 6/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0714 - accuracy: 0.9697
Epoch 7/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0599 - accuracy: 0.9737
Epoch 8/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0549 - accuracy: 0.9722
Epoch 9/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0499 - accuracy: 0.9769
Epoch 10/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0465 - accura

127/127 [==============================] - 1s 6ms/step - loss: 0.0075 - accuracy: 0.9946
Epoch 82/100
127/127 [==============================] - 1s 5ms/step - loss: 0.0085 - accuracy: 0.9936
Epoch 83/100
127/127 [==============================] - 1s 5ms/step - loss: 0.0081 - accuracy: 0.9943
Epoch 84/100
127/127 [==============================] - 1s 5ms/step - loss: 0.0076 - accuracy: 0.9941
Epoch 85/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0072 - accuracy: 0.9948
Epoch 86/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0074 - accuracy: 0.9946
Epoch 87/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0075 - accuracy: 0.9941
Epoch 88/100
127/127 [==============================] - 1s 7ms/step - loss: 0.0075 - accuracy: 0.9951
Epoch 89/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0078 - accuracy: 0.9953
Epoch 90/100
127/127 [==============================] - 1s 6ms/step - loss: 0.0095 - accuracy: 

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

43/43 - 0s - loss: 3.6971 - accuracy: 0.5365 - 216ms/epoch - 5ms/step
Loss: 3.6970620155334473, Accuracy: 0.536531388759613


In [23]:
# create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    model = tf.keras.models.Sequential()

    # allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=3930))

    # allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # compile the model
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return model

In [24]:
# import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

In [25]:
# run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 09s]
val_accuracy: 0.5682656764984131

Best val_accuracy So Far: 0.5926199555397034
Total elapsed time: 00h 05m 31s


In [26]:
# get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 5,
 'num_layers': 3,
 'units_0': 3,
 'units_1': 5,
 'units_2': 7,
 'units_3': 3,
 'units_4': 5,
 'units_5': 5,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [27]:
# evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

43/43 - 0s - loss: 0.6868 - accuracy: 0.5926 - 319ms/epoch - 7ms/step
Loss: 0.6867666244506836, Accuracy: 0.5926199555397034
